In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import os
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# from nltk.corpus import stopwords
# from nltk.stem import SnowballStemmer
# from nltk.stem import WordNetLemmatizer
import re  
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [27]:
import nltk
import ssl
 
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stops = set(stopwords.words("english"))
print(stops)

from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gayan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gayan\AppData\Roaming\nltk_data...


{'i', 'o', 'at', 'that', 'how', "didn't", 'hasn', 'our', 'themselves', 'no', 'most', 'than', 'on', 'were', 'she', 'against', 'm', "shouldn't", 'more', 'with', "wouldn't", 'own', 'but', 'ma', "aren't", 'it', 'needn', 'before', 'such', "don't", "doesn't", 'here', 'couldn', 'all', 'my', 'be', 'you', 'for', "weren't", "you'll", "haven't", 'this', 'because', "isn't", 'will', 'further', 'off', 'after', 'now', 'having', 'into', 'me', "that'll", 'was', 'is', "won't", 'shan', 'hers', 'her', 'whom', 'so', 'as', 'if', 'very', "wasn't", 'has', 're', 'herself', 'below', "you've", 'don', "needn't", 'aren', 'from', 'won', 've', 'him', 'ourselves', 'its', 'their', 'which', 'a', 'other', 'weren', 'why', 'do', 'mightn', "you'd", 'they', 'those', 'any', 'shouldn', 'during', 'few', 'can', 'over', 'mustn', 'doesn', 'wouldn', 'wasn', "hadn't", 'again', "mustn't", 'while', 'or', 'just', 'who', 'been', 'nor', 'there', 'have', 'these', 'had', 'up', 'down', 'being', 'some', 'we', 'itself', 'haven', 'ain', 'he',

[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [4]:
import pandas as pd

model_name = "test-data-15000"

df = pd.read_csv(model_name+'.csv')
df.head(10)

,key,product_l1,summary,description,comments
0,P10237313-17413,10049895,[VCF]formal core tgl coverage issue,Hi Penny This is the issue reproduced from MT...,Hi Mehr Per discussion with Penny please tak...
1,P10049895-134713,Verdi,Wrong Layer/PermName annotation of parameters ...,The below testcase's Layer/PermName annotation...,NaN
2,P10049895-134712,Verdi,Verdi hang when selecting Toggle metric pane f...,I have loaded a vdb in verdi and trying to exp...,NaN
3,P10049895-134710,Verdi,RTM: *ERROR* Assertion failed file ufedr_doeb...,/slowfs/swe158/rtm2outputs/kravoori/bugs/kdb/f...,Product team and Watchers are re-assigned due ...
4,P10049895-134708,Verdi,[ZWDG] End Tree is mssing,We find that FSDB_TREE_CBT_END_TREE is missing...,Chime Request: 968035 created for the stream T...
5,P10049895-134707,Verdi,Update the comment description of API hdbIsInt...,Update the comment description of API hdbIsInt...,NaN
6,P10049895-134706,Verdi,[Verdi][FPV] Escape name verilog blackbox inst...,Hi Team Escape name generate verilog black...,Product team and Watchers are re-assigned due ...
7,P10049895-134705,Verdi,[Vega2] can't get vhdl original case name on t...,Vega2 flow VHDL can't show original case top ...,Chime Request: 967717 created for the stream T...
8,P10049895-134704,Verdi,[VBT] Memory bug in vbt_dyn_ary_t,The reallocation conditions in vbt_dyn_ary_t::...,NaN
9,P10049895-134702,Verdi,RTM: assert @ sysCnlAssert(oh_handle.cpp:2445),|[bucket|https://peweb/remote/vgrtmlogs5/RTM2/...,Dear Kevin The assertion at oh_handle...


In [5]:
def view_df(df):
    print(f"Data shape: {df.shape}")
    print("Check for null values:")
    print(df.isnull().sum())
#     print("Check for duplicates:")
#     print(df.is_duplicate.value_counts())
    print(df.head(10))

In [6]:
view_df(df)

Data shape: (16633, 5)
Check for null values:
key               0
product_l1        0
summary           0
description       1
comments       1591
dtype: int64
                key product_l1  \
0   P10237313-17413   10049895   
1  P10049895-134713      Verdi   
2  P10049895-134712      Verdi   
3  P10049895-134710      Verdi   
4  P10049895-134708      Verdi   
5  P10049895-134707      Verdi   
6  P10049895-134706      Verdi   
7  P10049895-134705      Verdi   
8  P10049895-134704      Verdi   
9  P10049895-134702      Verdi   

                                             summary  \
0                [VCF]formal core tgl coverage issue   
1  Wrong Layer/PermName annotation of parameters ...   
2  Verdi hang when selecting Toggle metric pane f...   
3  RTM: *ERROR* Assertion failed  file ufedr_doeb...   
4                          [ZWDG] End Tree is mssing   
5  Update the comment description of API hdbIsInt...   
6  [Verdi][FPV] Escape name verilog blackbox inst...   
7  [Vega2] can't g

In [7]:
# Remove Null data
df.dropna(inplace=True)
view_df(df)

Data shape: (15041, 5)
Check for null values:
key            0
product_l1     0
summary        0
description    0
comments       0
dtype: int64
                 key product_l1  \
0    P10237313-17413   10049895   
3   P10049895-134710      Verdi   
4   P10049895-134708      Verdi   
6   P10049895-134706      Verdi   
7   P10049895-134705      Verdi   
9   P10049895-134702      Verdi   
10  P10049895-134699      Verdi   
15  P10049895-134693      Verdi   
16  P10049895-134689      Verdi   
17  P10049895-134688      Verdi   

                                              summary  \
0                 [VCF]formal core tgl coverage issue   
3   RTM: *ERROR* Assertion failed  file ufedr_doeb...   
4                           [ZWDG] End Tree is mssing   
6   [Verdi][FPV] Escape name verilog blackbox inst...   
7   [Vega2] can't get vhdl original case name on t...   
9      RTM: assert @ sysCnlAssert(oh_handle.cpp:2445)   
10  [NLP] Mistakenly try to find split-module of l...   
15            

In [8]:
df.columns

Index(['key', 'product_l1', 'summary', 'description', 'comments'], dtype='object')

In [9]:
# Create corpus from data frame
corpus = df.to_dict('records')
print(corpus[0:5])

[{'key': 'P10237313-17413', 'product_l1': '10049895', 'summary': '[VCF]formal core tgl coverage issue', 'description': "Hi Penny  This is the issue reproduced from MTK. If we simply assign a wire to 1: wire a = 1; And we write a fvassert : fvassert -expr {a!=0} Then compute the formal core results.  The generated coverage db will show both 0->1 and 1->0 toggle coverage of this signal 'a' is covered  which it shouldn't be.  Reproducer: /remote/testcases/TC12/9001512001_9001512500/9001512166/formal_core_tgl_bug vcf -f vcf.tcl -gui &", 'comments': 'Hi Mehr  Per discussion with Penny  please take a look at this item.Hi SJ    This is known and there is no immediate fix for this. This is related to the mechanism in which formal core registers are mapped to coverage. What we do is we find the related registers and mark the corresponding metrics accordingly. This is done using structural analysis and at that level there is no information regarding the value configuration of the registers which

In [10]:
def clean_text(text, remove_stopwords=True):
    # Clean the text, with the option to remove stopwords.
    
    # Convert words to lower case and split them
    words = text.lower().split()

    # Optionally remove stop words (true by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    cleaned_text = " ".join(words)

    # Clean the text
    cleaned_text = re.sub(r"[^A-Za-z0-9(),!.?\'\`]", " ", cleaned_text)
    cleaned_text = re.sub(r"\'s", " 's ", cleaned_text)
    cleaned_text = re.sub(r"\'ve", " 've ", cleaned_text)
    cleaned_text = re.sub(r"n\'t", " 't ", cleaned_text)
    cleaned_text = re.sub(r"\'re", " 're ", cleaned_text)
    cleaned_text = re.sub(r"\'d", " 'd ", cleaned_text)
    cleaned_text = re.sub(r"\'ll", " 'll ", cleaned_text)
    cleaned_text = re.sub(r",", " ", cleaned_text)
    cleaned_text = re.sub(r"\.", " ", cleaned_text)
    cleaned_text = re.sub(r"!", " ", cleaned_text)
    cleaned_text = re.sub(r"\(", " ( ", cleaned_text)
    cleaned_text = re.sub(r"\)", " ) ", cleaned_text)
    cleaned_text = re.sub(r"\?", " ", cleaned_text)
    cleaned_text = re.sub(r"\s{2,}", " ", cleaned_text)
    cleaned_text = re.sub(r"\b[0x]{0,2}[0-9a-fA-F]+\b", " ", cleaned_text)     # Remove numbers and hexadecimals
    
    words = re.split('\W+', cleaned_text)                        # remove all non-words (make a list)
#     words = cleaned_text.split()
    
    # Shorten words to their stems
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in words]
    
    cleaned_text = " ".join(stemmed_words)
    
    # Return cleaned text 
    return(cleaned_text)

# test clean_text
#_text="Verdi issue : stop cmd is not getting triggered when there is change in event"
#print(clean_text(_text))

In [11]:
# Input: corpus is a list of dictionaries(with all fields and data) 
def extract_clean_documents_from_corpus(corpus):
    print("Extracting and Cleaning documents...")
    final_corpus = []
    list_of_docs = []
    i = 0
    for ticket_dict in corpus:
#         print("Processing ",ticket_dict['summary'])
        doc_cleaned_text = ''
        document_of_words = (str(ticket_dict['summary'])+" "+str(ticket_dict['description']))
#         document_of_words = (str(ticket_dict['question1']))
        doc_cleaned_text = clean_text(document_of_words)
        list_of_docs.append(doc_cleaned_text)
        final_corpus.append({'key':ticket_dict['key'], 'words':doc_cleaned_text, 'index':i})
#         final_corpus.append({'qid1':ticket_dict['qid1'], 'words':doc_cleaned_text, 'index':i})
        i+=1
    return list_of_docs,final_corpus



In [12]:
# extract_clean_documents_from_corpus
list_of_docs,training_ticket_corpus = extract_clean_documents_from_corpus(corpus)

Extracting and Cleaning documents...


In [14]:
training_ticket_corpus

[{'key': 'P10237313-17413',
  'words': ' vcf formal core tgl coverag issu hi penni issu reproduc mtk simpli assign wire wire write fvassert fvassert expr comput formal core result generat coverag show toggl coverag signal cover reproduc remot testcas tc12 formal core tgl bug vcf vcf tcl gui ',
  'index': 0},
 {'key': 'P10049895-134710',
  'words': 'rtm error assert fail file ufedr doeb oh cpp line slowf swe158 rtm2output kravoori bug kdb fail ufedr doeb oh cpp us01odcvde17897 run csh error assert fail file ufedr doeb oh cpp line syscnlassert ufedr doeb oh cpp elabcom crash intern error tool s sourc file kdb builder line assert fail pleas send messag vcs support synopsi com ',
  'index': 1},
 {'key': 'P10049895-134708',
  'words': ' zwdg end tree mssing find fsdb tree cbt end tree miss tree callback ',
  'index': 2},
 {'key': 'P10049895-134706',
  'words': ' verdi fpv escap name verilog blackbox instanc like core u sub1 highlight verdi instanc hierarchi view hi team escap name generat v

In [15]:
tagged_data = [TaggedDocument(words=str(ticket_dict['words']), tags=[str(ticket_dict['key'])]) for ticket_dict in training_ticket_corpus]
tagged_data

[TaggedDocument(words=' vcf formal core tgl coverag issu hi penni issu reproduc mtk simpli assign wire wire write fvassert fvassert expr comput formal core result generat coverag show toggl coverag signal cover reproduc remot testcas tc12 formal core tgl bug vcf vcf tcl gui ', tags=['P10237313-17413']),
 TaggedDocument(words='rtm error assert fail file ufedr doeb oh cpp line slowf swe158 rtm2output kravoori bug kdb fail ufedr doeb oh cpp us01odcvde17897 run csh error assert fail file ufedr doeb oh cpp line syscnlassert ufedr doeb oh cpp elabcom crash intern error tool s sourc file kdb builder line assert fail pleas send messag vcs support synopsi com ', tags=['P10049895-134710']),
 TaggedDocument(words=' zwdg end tree mssing find fsdb tree cbt end tree miss tree callback ', tags=['P10049895-134708']),
 TaggedDocument(words=' verdi fpv escap name verilog blackbox instanc like core u sub1 highlight verdi instanc hierarchi view hi team escap name generat verilog blackbox instanc get highl

In [ ]:
#save and load moddel
def save_model(model,model_name):
    pickle.dump(model, open(model_name, "wb"))

def load_model(model_file_path):
    with open(model_file_path, 'rb') as pickled_file:
        loaded_model_data = pickle.load(pickled_file)
    return loaded_model_data

<b>Train the model</b>

In [18]:
# Train the model
def train_doc2vec_model(model_name_prefix,tagged_data, vec_size, alpha, min_word_count_per_doc, dm, no_of_epochs):
    model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=alpha,
                min_count=min_word_count_per_doc,
                dm =dm,
                workers = 4)
    
    model.build_vocab(tagged_data)
    
#     for epoch in range(no_of_epochs):
#         model.train(tagged_data,
#                 total_examples=model.corpus_count,
#                 epochs=model.iter)
#         # decrease the learning rate
#         model.alpha -= 0.0002
#         # fix the learning rate, no decay
#         model.min_alpha = model.alpha
    model.train(tagged_data,total_examples=model.corpus_count,epochs=no_of_epochs)
    modelname = model_name_prefix+"_d2v_"+str(dm)+"dm_"+str(no_of_epochs)+"epoch_"+str(vec_size)+"vecsize_"+str(alpha)+"alpha.model"
    model.save(modelname)
    print("Model trained:",modelname)

# save_model(tfidf_trainingset,model_name+".pickle")
# save_model(training_ticket_corpus,model_name+"_corpus.pickle")

In [19]:
# epochs_list = [100,500,1000]
epochs_list = [10,50,100]
size_list = [10,50,100,200]
alpha_list = [0.05, 0.01, 0.025]
dm_list = [0,1]

i = 1
for vec_size in size_list:
    for alpha in alpha_list:
        for epoch in epochs_list:
            for dm in dm_list:
                train_doc2vec_model(str(i),tagged_data, vec_size, alpha, 1, dm, epoch)
                i += 1

Model trained: 1_d2v_0dm_10epoch_10vecsize_0.05alpha.model
Model trained: 2_d2v_1dm_10epoch_10vecsize_0.05alpha.model
Model trained: 3_d2v_0dm_50epoch_10vecsize_0.05alpha.model
Model trained: 4_d2v_1dm_50epoch_10vecsize_0.05alpha.model
Model trained: 5_d2v_0dm_100epoch_10vecsize_0.05alpha.model
Model trained: 6_d2v_1dm_100epoch_10vecsize_0.05alpha.model
Model trained: 7_d2v_0dm_10epoch_10vecsize_0.01alpha.model
Model trained: 8_d2v_1dm_10epoch_10vecsize_0.01alpha.model
Model trained: 9_d2v_0dm_50epoch_10vecsize_0.01alpha.model
Model trained: 10_d2v_1dm_50epoch_10vecsize_0.01alpha.model
Model trained: 11_d2v_0dm_100epoch_10vecsize_0.01alpha.model
Model trained: 12_d2v_1dm_100epoch_10vecsize_0.01alpha.model
Model trained: 13_d2v_0dm_10epoch_10vecsize_0.025alpha.model
Model trained: 14_d2v_1dm_10epoch_10vecsize_0.025alpha.model
Model trained: 15_d2v_0dm_50epoch_10vecsize_0.025alpha.model
Model trained: 16_d2v_1dm_50epoch_10vecsize_0.025alpha.model
Model trained: 17_d2v_0dm_100epoch_10vecs

KeyboardInterrupt: 

In [20]:
result_df = pd.DataFrame({'Modelname':['TestModel'], 'For':['TEST-123'], 'Related-Tickets':['TEST;TEST'], 'Points':[-10]})
result_df

,Modelname,For,Related-Tickets,Points
0,TestModel,TEST-123,TEST;TEST,-10


['10_d2v_1dm_50epoch_10vecsize_0.01alpha.model',
 '11_d2v_0dm_100epoch_10vecsize_0.01alpha.model',
 '12_d2v_1dm_100epoch_10vecsize_0.01alpha.model',
 '13_d2v_0dm_10epoch_10vecsize_0.025alpha.model',
 '14_d2v_1dm_10epoch_10vecsize_0.025alpha.model',
 '15_d2v_0dm_50epoch_10vecsize_0.025alpha.model',
 '16_d2v_1dm_50epoch_10vecsize_0.025alpha.model',
 '17_d2v_0dm_100epoch_10vecsize_0.025alpha.model',
 '18_d2v_1dm_100epoch_10vecsize_0.025alpha.model',
 '19_d2v_0dm_10epoch_50vecsize_0.05alpha.model',
 '1_d2v_0dm_10epoch_10vecsize_0.05alpha.model',
 '20_d2v_1dm_10epoch_50vecsize_0.05alpha.model',
 '21_d2v_0dm_50epoch_50vecsize_0.05alpha.model',
 '22_d2v_1dm_50epoch_50vecsize_0.05alpha.model',
 '23_d2v_0dm_100epoch_50vecsize_0.05alpha.model',
 '24_d2v_1dm_100epoch_50vecsize_0.05alpha.model',
 '25_d2v_0dm_10epoch_50vecsize_0.01alpha.model',
 '26_d2v_1dm_10epoch_50vecsize_0.01alpha.model',
 '27_d2v_0dm_50epoch_50vecsize_0.01alpha.model',
 '28_d2v_1dm_50epoch_50vecsize_0.01alpha.model',
 '29_d2v_

<b>Load models and get result<b>

In [30]:
tickets_dev_set = corpus[1000:1005]
models_list = os.listdir('./models')
for modelname in models_list:
    print("processing ",modelname,"....")
    model = Doc2Vec.load("./models/"+str(modelname))
    for ticket in tickets_dev_set:
        data_dict = {}
        test_issue_text_data = ''
        points = 0
        data_dict['Modelname'] = modelname
        data_dict['Points'] = 0

        # ['key', 'product_l1', 'summary', 'description', 'comments'
        data_dict['For'] = ticket['key']
        
        test_issue_text_data += str(ticket['summary'])
        test_issue_text_data += str(ticket['description'])
        
        cleaned_test_issue_text_data = clean_text(test_issue_text_data)
        test_data = word_tokenize(cleaned_test_issue_text_data.lower())
        test_issue_vector = model.infer_vector(test_data)
        similar_doc = model.dv.most_similar(positive=[test_issue_vector], topn=5)
        data_dict['Related-Tickets'] = ":".join(["("+str(doc)+"|"+str("{0:.3f}".format(similar_score))+")" for (doc,similar_score) in similar_doc])
        for (doc,similar_score) in similar_doc:
            points+=float(similar_score)
        data_dict['Points'] = points
        result_df.loc[result_df.index.max() + 1] = data_dict
    print("all done for model - ",modelname)

processing  10_d2v_1dm_50epoch_10vecsize_0.01alpha.model ....
all done for model -  10_d2v_1dm_50epoch_10vecsize_0.01alpha.model
processing  11_d2v_0dm_100epoch_10vecsize_0.01alpha.model ....
all done for model -  11_d2v_0dm_100epoch_10vecsize_0.01alpha.model
processing  12_d2v_1dm_100epoch_10vecsize_0.01alpha.model ....
all done for model -  12_d2v_1dm_100epoch_10vecsize_0.01alpha.model
processing  13_d2v_0dm_10epoch_10vecsize_0.025alpha.model ....
all done for model -  13_d2v_0dm_10epoch_10vecsize_0.025alpha.model
processing  14_d2v_1dm_10epoch_10vecsize_0.025alpha.model ....
all done for model -  14_d2v_1dm_10epoch_10vecsize_0.025alpha.model
processing  15_d2v_0dm_50epoch_10vecsize_0.025alpha.model ....
all done for model -  15_d2v_0dm_50epoch_10vecsize_0.025alpha.model
processing  16_d2v_1dm_50epoch_10vecsize_0.025alpha.model ....
all done for model -  16_d2v_1dm_50epoch_10vecsize_0.025alpha.model
processing  17_d2v_0dm_100epoch_10vecsize_0.025alpha.model ....
all done for model - 

In [31]:
result_df

,Modelname,For,Related-Tickets,Points
0,TestModel,TEST-123,TEST;TEST,-10.000000
1,10_d2v_1dm_50epoch_10vecsize_0.01alpha.model,P10237313-17413,(P10049895-107401|0.912):(P10049895-114135|0.9...,4.477212
2,10_d2v_1dm_50epoch_10vecsize_0.01alpha.model,P10049895-134710,(P10049895-122823|0.486):(P10049895-108529|0.4...,2.406775
3,10_d2v_1dm_50epoch_10vecsize_0.01alpha.model,P10049895-134708,(P10049895-114605|0.891):(P10049895-131019|0.8...,4.271526
4,10_d2v_1dm_50epoch_10vecsize_0.01alpha.model,P10049895-134706,(P10049895-114581|0.608):(P10049895-107376|0.5...,2.819395
...,...,...,...,...
456,9_d2v_0dm_50epoch_10vecsize_0.01alpha.model,P10049895-132629,(P10049895-131000|0.779):(P10049895-129237|0.7...,3.649441
457,9_d2v_0dm_50epoch_10vecsize_0.01alpha.model,P10049895-132628,(P10049895-130769|0.930):(P10049895-133301|0.9...,4.546313
458,9_d2v_0dm_50epoch_10vecsize_0.01alpha.model,P10049895-132626,(P10049895-104169|0.862):(P10049895-104172|0.8...,4.214222
459,9_d2v_0dm_50epoch_10vecsize_0.01alpha.model,P10049895-132623,(P10049895-109314|0.550):(P10049895-121154|0.5...,2.548568


<b>Model with highest similarity score <b>

In [34]:
result_df[result_df['Points']==result_df['Points'].max()]

,Modelname,For,Related-Tickets,Points
447,7_d2v_0dm_10epoch_10vecsize_0.01alpha.model,P10049895-132628,(P10049895-98400|0.975):(P10049895-129417|0.97...,4.847418


In [40]:
def find_top_n_similar_documents_doc2vec(n,test_doc,model):
    test_data = word_tokenize(test_doc.lower())
    test_issue_vector = model.infer_vector(test_data)
    similar_doc = model.dv.most_similar(positive=[test_issue_vector], topn=n)
    return similar_doc

In [42]:

# my_model = load_model(model_name+".pickle")
# my_corpus = load_model(model_name+"_corpus.pickle")
# modelname = '7_d2v_0dm_10epoch_10vecsize_0.01alpha.model'
modelname = '46_d2v_1dm_50epoch_100vecsize_0.01alpha.model'
model = Doc2Vec.load("./models/"+str(modelname))
tickets_dev_set = corpus[0:5]

for ticket in tickets_dev_set:
    
    document_test = (str(ticket['summary'])+" "+str(ticket['description']))
    cleaned_document = clean_text(document_test)
    similar_doc = find_top_n_similar_documents_doc2vec(10,cleaned_document,model)
    print("\n",ticket['key'],">>>> \n")
    [print(doc," : ",similar_score,"\n") for (doc,similar_score) in similar_doc]


 P10237313-17413 >>>> 

P10049895-134464  :  0.29888206720352173 

P10049895-119139  :  0.29583874344825745 

P10049895-121821  :  0.27222925424575806 

P10049895-132279  :  0.26705285906791687 

P10049895-97875  :  0.2662534713745117 

P10049895-125951  :  0.26445668935775757 

P10049895-125284  :  0.25261515378952026 

P10049895-129562  :  0.2510317265987396 

P10049895-96345  :  0.24935483932495117 

P10049895-123985  :  0.2471797615289688 


 P10049895-134710 >>>> 

P10049895-114318  :  0.35885435342788696 

P10049895-96486  :  0.33727121353149414 

P10049895-128377  :  0.2979736328125 

P10049895-131447  :  0.2967644929885864 

P10049895-131504  :  0.2938893437385559 

P10049895-131390  :  0.29192209243774414 

P10049895-106049  :  0.2877580523490906 

P10049895-113400  :  0.2870788872241974 

P10049895-133976  :  0.27292677760124207 

P10049895-113838  :  0.27107009291648865 


 P10049895-134708 >>>> 

P10049895-131284  :  0.3011399209499359 

P10049895-103866  :  0.296479910612

In [ ]:
print(tfidf_test)